In [142]:
file_name = '1_all_1.csv'

In [143]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import re
from pandas import DataFrame, Series
import pickle

In [144]:
temp = pd.read_csv(file_name)

In [145]:
temp.shape

(226328, 64)

In [146]:
temp.index = temp['Unnamed: 0']

In [147]:
temp.drop('Unnamed: 0', axis=1, inplace=True)

In [148]:
temp.drop('address', axis=1, inplace=True)

In [149]:
temp.drop('MLSAreaMajor', inplace=True, axis=1)

In [150]:
temp.drop('rid', axis=1, inplace=True)

In [151]:
temp.drop('City', axis=1, inplace=True)

In [152]:
done = temp.loc[:,['sold_date', 'price', 'detail_url', 'city', 'zipcode']]
homes_df = DataFrame(done)

In [153]:
ratio = temp.count() * 1.0 / len(temp)
print ratio.sort_values()

LaundryFeatures             0.096700
RoomType                    0.103673
StreetSuffix                0.107410
PreviousStandardStatus      0.116079
YearBuiltSource             0.116406
LotSizeSource               0.118147
ElevationUnits              0.120374
LotFeatures                 0.122711
LivingAreaUnits             0.123573
WaterSource                 0.125455
HighSchoolDistrict          0.125477
StateOrProvince             0.125915
StandardStatus              0.125915
SpecialListingConditions    0.126816
CommunityFeatures           0.127443
Amenities                   0.132326
Levels                      0.132661
Unit floor #                0.142112
CommonWalls                 0.142643
Middle school               0.148992
Materials                   0.156057
PropertyType                0.161871
Elementary school           0.168097
High school                 0.181851
Baths                       0.199109
Yard                        0.220326
View                        0.250941
H

# Type and Style

In [154]:
temp_new = pd.get_dummies(temp, columns=['Type and Style'])

In [155]:
homes_df[temp_new.columns[-10:]] = temp_new.iloc[:, -10:]

# size

In [156]:
homes_df['size'] = temp['size']

# Floor size

In [157]:
temp['Floor size'][:6]

Unnamed: 0
1    1,057 sqft
2      680 sqft
3    1,188 sqft
4    1,266 sqft
5    1,198 sqft
6    1,500 sqft
Name: Floor size, dtype: object

In [158]:
temp.drop('Floor size', axis=1, inplace=True)

# Beds

In [159]:
temp.Beds.describe()

count    226328.000000
mean          3.065034
std           0.977344
min           1.000000
25%           2.000000
50%           3.000000
75%           4.000000
max          12.000000
Name: Beds, dtype: float64

In [160]:
homes_df['Beds'] = temp.Beds

# Dates

In [161]:
date = temp.Dates.dropna()

In [162]:
date = date.map(lambda x: int(x[-6: -2]))

In [163]:
homes_df['built_date'] = date

# Unit count

## Waiting: 不懂含意 数值情况很奇怪

In [164]:
to_do = ['Unit count']

In [165]:
homes_df['unit_count'] = temp['Unit count']

# Lot

## Waiting: 异常值

In [166]:
lot = temp.Lot.dropna()

In [167]:
def cleanLot(x):
    c =x.split(' ')
    num = float(filter(str.isdigit, c[0]))
    if c[1] == 'sqft':
        return num
    elif c[1].find('acre') != -1:
        return 43560 * num
    else:
        print x
        return np.nan

In [168]:
lot = lot.map(cleanLot)

In [169]:
homes_df['lot'] = lot

In [170]:
to_do.append('lot')

# Parking

In [171]:
parking = temp.Parking.dropna()

In [172]:
def cleanParkingSpace(x):
    c = x.split(', ')
    has_num = False
    for cc in c:
        matchObj = re.match('(\d+) sqft.*', cc)
        if matchObj:
            has_num = True
            return int(matchObj.group(1))
    if has_num == False:
        return np.nan

In [173]:
space = parking.map(cleanParkingSpace)

In [174]:
def cleanParkingNum(x):
    c = x.split(', ')
    has_num = False
    for cc in c:
        matchObj = re.match('(\d+) space.*', cc)
        if matchObj:
            has_num = True
            return int(matchObj.group(1))
    if has_num == False:
        return np.nan

In [175]:
num = parking.map(cleanParkingNum)

In [176]:
homes_df['garage_num'] = num

# Stories

In [177]:
import types
def cleanStories(x):
    if type(x) is types.StringType:
        c = x.split()
        if str.isdigit(c[0]):
            num = int(c[0])
            if num == 0:
                return 1
            else: 
                return num
        else:
            if c[0] == 'One':
                return 1
            elif c[0] == 'Upper':
                return 1
            elif c[0] == 'Tri-Level':
                return 3
            elif re.match('(\d+)(\.\d+)*', x):
                matchObj = re.match('(\d+)\.\d+', x)
                num = int(matchObj.group(1))
                if num == 0:
                    return 1
                else:
                    return num
            else:
                print c
                return 0
    else:
        return int(x)

In [178]:
stories = temp.Stories.dropna()

In [179]:
stories[stories == 0] = 1

In [180]:
stories_new = stories.map(cleanStories)

In [181]:
stories[stories_new == 225]

Unnamed: 0
16617    225
Name: Stories, dtype: object

In [182]:
stories_new[stories_new == 225] = np.nan

In [183]:
homes_df['stories'] = stories_new

# Heating

In [184]:
from collections import defaultdict
heating = temp.Heating.dropna()
s = defaultdict(int)
for i in range(len(heating)):
    items = heating.iloc[i].split(', ')
    for item in items:
        if item == 'Kerosene':
            print i 
        s[item] += 1
print s

175397
defaultdict(<type 'int'>, {'Forced air': 23072, 'Wall Furnace': 2, 'Natural Gas': 179, 'Heating - 2+ Zones': 2, 'Gravity': 43, 'Central Forced Air': 4, 'Wall Gas': 4, 'Other': 50656, 'Energy Star': 39, 'High Efficiency': 70, 'See Remarks': 164, 'Heat Pump': 9, 'Oil': 14, 'Electric': 4418, 'Combination': 37, 'Propane': 9, 'Kerosene': 1, 'Gas': 31130, 'Baseboard': 180, 'Hot Water Circulator': 2, 'Solar': 874, 'Wall Heater': 1707, 'Fireplace': 271, 'Cent Forced Air': 2, 'Woodstove': 3, 'Heat pump': 361, 'None': 27407, 'Central': 5647, 'Propane Gas': 1, 'Central Forced Heat': 1, 'Pellet Stove': 1, 'Radiant': 678, 'Stove - Wood': 1, 'Coal': 13, 'Floor Furnace': 746, 'Space Heater': 11, 'Solar Heat Passive': 1, 'Wood Burn. Stove': 1, 'FAG': 1, 'Central Forced Air - Gas': 4, 'Wall Electric': 2, 'Wood Stove': 11, 'Humidity Control': 3, 'Forced Air': 320, 'Propane butane': 176, 'Zoned': 104, 'Wall': 20216, 'Pellet': 6, 'Stove': 249, 'Natural Gas Heat': 1, 'Dual Pack': 1, 'Propane Wall': 

In [185]:
# air heating system:'Forced air', 'Central Forced Air' ,'Cent Forced Air', 'Forced Air', 
##'Central Forced Air - Gas'

# Furnace:'Wall Furnace' 'Fireplace' 'Floor Furnace','Central Furnace'

# Gas: 'Natural Gas', 'Wall Gas', 'Gas', 'Central Forced Air - Gas', 'Natural Gas Heat'

# 'Gravity', 'Gravity Heating'

#'Heat Pump' 'Heat pump'

# 'Oil'

# 'Electric', 'Wall Electric'

# propane: 丙烷
## 'Propane', 'Propane Gas', 'Propane butane', 'Propane Wall'

# butane: 丁烷 'Propane butane'

# 'Kerosene'：煤油'Kerosene'

# 'Baseboard'

# 'Solar', 'Solar Heat Passive', 'Solar Heat Other'

# stove:'Woodstove', 'Pellet Stove', 'Stove - Wood', 'Wood Burn. Stove', 'Wood Stove', 
##'Wood', 'Wood / pellet', 'pellet', Pellet, Stove'

# 'Radiant'

# 'Coal'

# 'Energy Star' ,'High Efficiency'
#---------------------------------------------------------------------------------------
# 'Zoned', 'Heating - 2+ Zones'

# Cental: 'Central', 'Central Furnace', 'Central Forced Air', 'Central Forced Heat','Central Forced Air - Gas'

# Floor: 'Floor Furnace', Baseboard'

# Wall: 'Wall Heater', 'Wall Electric', ‘Wall', 'Propane Wall', 'Wall Furnace', 'Wall Gas'

# 'Combination'

# 'None'

#'Other', 'Solar Heat Other' 'Humidity Control', 'FAG', 'Space Heater', 'Hot Water Circulator'
heating_system = ['air', 'furnace', 'gas', 'gravity', 'pump', 'oil', 'electric', 'propane', 'kerosene', 'baseboard',
                 'solar', 'stove', 'radiant', 'coal', 'high_efficiency', 
                 'zoned', 'central', 'floor', 'wall', 'combination', 'none', 'other']

heating_system = {
    'air' : ['Forced air', 'Central Forced Air' ,'Cent Forced Air', 'Forced Air', 'Central Forced Air - Gas'],
    'furnace' : ['Wall Furnace' 'Fireplace' 'Floor Furnace','Central Furnace'],
    'gas' : ['Natural Gas', 'Wall Gas', 'Gas', 'Central Forced Air - Gas', 'Natural Gas Heat'],
    'gravity' : ['Gravity', 'Gravity Heating'],
    'pump' : ['Heat Pump','Heat pump'],
    'oil' : ['Oil'],
    'electric' : ['Electric', 'Wall Electric'],
    'propane' : ['Propane', 'Propane Gas', 'Propane butane', 'Propane Wall'],
    'kerosene' : ['Kerosene'],
    'baseborad' : ['Baseboard'],
    'solar' : ['Solar', 'Solar Heat Passive', 'Solar Heat Other'],
    'stove' : ['Woodstove', 'Pellet Stove', 'Stove - Wood', 'Wood Burn. Stove', 'Wood Stove', 
         'Wood', 'Wood / pellet', 'pellet', 'Pellet', 'Stove'],
    'radiant' : ['Radiant'],
    'coal' : ['Coal'],
    'high_efficiency' : ['Energy Star' ,'High Efficiency'],
    'zoned' : ['Zoned', 'Heating - 2+ Zones'],
    'central' : ['Central', 'Central Furnace', 'Central Forced Air', 'Central Forced Heat','Central Forced Air - Gas'],
    'floor' : ['Floor Furnace', 'Baseboard'],
    'wall' : ['Wall Heater', 'Wall Electric', 'Wall', 'Propane Wall', 'Wall Furnace', 'Wall Gas'],
    'combination' : ['Combination'],
    'none' : ['None'],
    'other' : ['Other', 'Hot Water Circulator', 'FAG', 'Humidity Control', 'Solar Heat Other', 'Space Heater']}

In [186]:
heating_df = DataFrame()
for heating_way in heating_system.keys():
    def cleanheating(x):
        c = x.split(', ')
        for cc in c:
            if cc in heating_system[heating_way]:
                return 1
        return 0
                      
    heating_new = heating.map(cleanheating)
    heating_df['heat_' + heating_way] = heating_new


In [187]:
homes_df[heating_df.columns] = heating_df

In [188]:
homes_df.shape

(226328, 44)

In [189]:
homes_df[heating_df.columns] = homes_df[heating_df.columns].fillna(0)

In [190]:
homes_df.drop('heat_none', axis=1, inplace=True)

# Cooling

In [191]:
cooling = temp.Cooling.dropna()
s_cooling = defaultdict(int)
for i in range(len(cooling)):
    items = cooling.iloc[i].split(', ')
    for item in items:
        s_cooling[item] += 1
print s_cooling

defaultdict(<type 'int'>, {'Ceiling Fan': 13, 'Central Air': 1, 'Multi/Zone': 62, 'Evaporative': 915, 'Attic Fan': 17, 'Central Forced Air': 6, 'Other': 4887, 'Refrigeration': 640, 'Swamp Cooler': 1, 'Energy Star': 96, 'Whole House Fan': 131, 'See Remarks': 828, 'Multi-Zone': 1, 'Dual Pack': 1, 'Electric': 533, 'Gas': 320, 'Air Conditioning': 1, 'High Efficiency': 143, 'Geothermal': 18, 'Whole House / Attic Fan': 1, 'SEER Rated 13-15': 41, 'Solar': 1145, 'Central Forced Air - Gas': 3, 'Window Air': 1, 'Evaporative Cooler': 1, 'None': 21761, 'Central': 125264, 'SEER Rated 16+': 17, 'Refrigerated Air': 1, 'Swamp Cooler(s)': 12, 'Humidity Control': 1, 'Zoned': 6, 'Wall': 5006, 'Window Unit(s)': 72, 'Room Refrig Air': 1, 'Dual': 351, 'Wall Window': 7})


In [192]:
cooling_system = {
    'fan': ['Ceiling Fan', 'Attic Fan', 'Whole House Fan', 'Whole House / Attic Fan'],
    'air': ['Central Forced Air', 'Central Forced Air - Gas'  ],
    'zone': ['Multi/Zone', 'Multi-Zone', 'Zoned'],
    'evaporative': ['Evaporative',  'Evaporative Cooler'],
    'other': ['Other'],
    'air_conditioning': ['Central Air', 'Refrigeration', 'Refrigerated Air', 
                         'Room Refrig Air', 'Window Air', 'Air Conditioning'],
    'swamp': ['Swamp Cooler', 'Swamp Cooler(s)'],
    'high_efficiency': ['Energy Star', 'High Efficiency'],
    'electric': ['Electric'],
    'gas': ['Gas'],
    'geothermal': ['Geothermal'],
    'solar': ['Solar'],
    'central': ['Central', 'Central Air', 'Central Forced Air', 'Central Forced Air - Gas'],
    'wall': ['Wall', 'Wall Window'],
}

In [193]:
cooling_df = DataFrame()
for cooling_way in cooling_system.keys():
    def cleanCooling(x):
        c = x.split(', ')
        for cc in c:
            if cc in cooling_system[cooling_way]:
                return 1
        return 0
                      
    cooling_new = cooling.map(cleanCooling)
    cooling_df['cool_' + cooling_way] = cooling_new


In [194]:
homes_df[cooling_df.columns] = cooling_df

In [195]:
homes_df[cooling_df.columns] = homes_df[cooling_df.columns].fillna(0)

# Last remodel year

In [196]:
last = temp['Last remodel year']
date = homes_df['built_date']
last = last.combine_first(date)

In [197]:
homes_df['last_remodel_year'] = last

# Flooring

In [198]:
flooring = temp.Flooring.dropna()

In [199]:
s_flooring = defaultdict(int)
for i in range(len(flooring)):
    items = flooring.iloc[i].split(', ')
    for item in items:
        s_flooring[item] += 1
print s_flooring

defaultdict(<type 'int'>, {'Partially Carpeted': 84, 'Wood Under Carpet': 30, 'Terrazzo': 23, 'Pavers': 24, 'Sheet Vinyl': 53, 'Parquet Wood': 27, 'Parquet': 29, 'Wood': 1863, 'Cement': 91, 'Tile': 56481, 'Granite': 39, 'Brick': 35, 'Linoleum': 171, 'See Remarks': 460, 'Stained Concrete': 3, 'Concrete Slab': 19, 'Ceramic Tile': 819, 'Slate': 1315, 'Wall-To-Wall Carpet': 41, 'Stone/Travertine': 157, 'Stone Tile': 155, 'Vinyl': 286, 'Stone': 1008, 'Travertine': 157, 'Laminate': 28509, 'Concrete': 1791, 'Vinyl/Linoleum': 246, 'Clay': 9, 'Mixed': 839, 'Wood Laminate': 34, 'Bamboo': 487, 'Marble': 180, 'Adobe': 1, 'Vinyl Tile': 89, 'Softwood': 665, 'Hardwood': 52900, 'Linoleum / Vinyl': 8149, 'Laminated': 505, 'Other': 17221, 'Pebble/Rock': 2, 'Wood/Wood Like': 761, 'Carpet': 62150})


In [200]:
flooring_system = {
    'carpet' : ['Partially Carpeted', 'Wood Under Carpet', 'Wall-To-Wall Carpet', 'Carpet'],
    'linoleum' : ['Linoleum'], #油布
    
    'wood' : ['Wood Under Carpet', 'Parquet Wood','Parquet', 'Wood', 'Wood Laminate', 'Softwood', 'Hardwood', 'Wood/Wood Like'],
    'softwood': ['Softwood'],
    'hardwood': ['Hardwood'],
    
    'paver' : ['Pavers'], # 沥青
    'cement' : ['Cement'],  # 水泥
    'concerete' : ['Stained Concrete', 'Concrete Slab', 'Concrete'], # 混凝土
    'bamboo':['Bamboo'],
    'sheet_vinyl' : ['Sheet Vinyl', 'Vinyl', 'Vinyl/Linoleum', 'Vinyl Tile', 'Linoleum / Vinyl'],
    'laminate': ['Laminate', 'Wood Laminate', 'Laminated'], #层压板
    
    'brick' : ['Brick','Clay', 'Adobe' ], # 砖
    'terrazzo' : ['Terrazzo'],  # 水磨石
    'granite' : ['Granite'], # 花岗岩
    'marble': ['Marble'], #大理石
    'slate': ['Slate', 'Stone/Travertine', 'Stone Tile', 'Stone', 'Travertine'], #石板
    'ceramic': ['Ceramic Tile'], #瓷
    
    'tile' : ['Tile', 'Ceramic Tile', 'Stone Tile', 'Vinyl Tile'],
    
    'mixed': ['Mixed'],
    'other': ['Other']
    
}

In [201]:
flooring_df = DataFrame()
for flooring_way in flooring_system.keys():
    def cleanFlooring(x):
        c = x.split(', ')
        for cc in c:
            if cc in flooring_system[flooring_way]:
                return 1
        return 0
                      
    flooring_new = flooring.map(cleanFlooring)
    flooring_df['floor_' + flooring_way] = flooring_new


In [202]:
homes_df[flooring_df.columns] = flooring_df

In [203]:
homes_df[flooring_df.columns] = homes_df[flooring_df.columns].fillna(0)

# Appliances included

In [204]:
appliance = temp['Appliances included'].dropna()
s_appliance = defaultdict(int)
for i in range(len(appliance)):
    items = appliance.iloc[i].split(', ')
    for item in items:
        s_appliance[item] += 1
print s_appliance

defaultdict(<type 'int'>, {'Dishwasher': 95644, 'Freezer': 12691, 'Microwave': 59020, 'Garbage disposal': 74474, 'Dryer': 30107, 'Refrigerator': 50610, 'Trash compactor': 5105, 'Washer': 32208, 'Range / Oven': 79589})


In [205]:
appliance_system = ['Dishwasher', 'Freezer', 'Microwave', 'Garbage disposal', 'Dryer', 'Refrigerator', 
                    'Trash compactor', 'Washer', 'Range / Oven']

In [206]:
appliance_df = DataFrame()
for appliance_way in appliance_system:
    def cleanAppliance(x):
        c = x.split(', ')
        for cc in c:
            if cc == appliance_way:
                return 1
        return 0
                      
    appliance_new = appliance.map(cleanAppliance)
    appliance_df['appliance_' + appliance_way] = appliance_new

In [207]:
homes_df[appliance_df.columns] = appliance_df
homes_df[appliance_df.columns] = homes_df[appliance_df.columns].fillna(0)

# Other Interior Features 

In [208]:
other = temp['Other Interior Features'].dropna()
s_other = defaultdict(int)
for i in range(len(other)):
    items = other.iloc[i][1:-1].split(',')
    for item in items:
        s_other[item] += 1
print s_other

defaultdict(<type 'int'>, {'"Storage"': 23122, '"Vaulted Ceiling"': 25217, '"Fireplace"': 95050, '"Furnished"': 996, '"Ceiling Fan"': 39676})


In [209]:
other_system = ['"Storage"','"Vaulted Ceiling"', '"Fireplace"', '"Furnished"','"Ceiling Fan"' ]

In [210]:
other_df = DataFrame()
for other_way in other_system:
    def cleanother(x):
        c = x[1:-1].split(',')
        for cc in c:
            if cc == other_way:
                return 1
        return 0
                      
    other_new = other.map(cleanother)
    other_df['other_' + other_way] = other_new

In [211]:
homes_df[other_df.columns] = other_df
homes_df[other_df.columns] = homes_df[other_df.columns].fillna(0)

# Roof type

In [212]:
roof = temp['Roof type'].dropna()
s_roof = defaultdict(int)
for i in range(len(roof)):
    item = roof.iloc[i]
    s_roof[item] += 1
print s_roof

defaultdict(<type 'int'>, {'Asphalt': 5391, 'Built up': 226, 'Slate': 497, 'Metal': 660, 'Shake Shingle': 6122, 'Other': 43902, 'Tile': 24644, 'Composition': 24189})


In [213]:
roof_system = ['Asphalt','Slate', 'Metal', 'Shake Shingle', 'Tile', 'Composition']

In [214]:
roof_df = DataFrame()
for roof_way in roof_system:
    def cleanroof(x):
        if x == roof_way:
            return 1
        return 0
                      
    roof_new = roof.map(cleanroof)
    roof_df['roof_' + roof_way] = roof_new

In [215]:
homes_df[roof_df.columns] = roof_df
homes_df[roof_df.columns] = homes_df[roof_df.columns].fillna(0)
homes_df.describe()

,price,zipcode,"Type and Style_[""Apartment""]","Type and Style_[""Condo""]","Type and Style_[""Cooperative""]","Type and Style_[""Miscellaneous""]","Type and Style_[""Mobile / Manufactured""]","Type and Style_[""Multi Family""]","Type and Style_[""Multiple Occupancy""]","Type and Style_[""Other""]",...,"other_""Vaulted Ceiling""","other_""Fireplace""","other_""Furnished""","other_""Ceiling Fan""",roof_Asphalt,roof_Slate,roof_Metal,roof_Shake Shingle,roof_Tile,roof_Composition
count,2.263280e+05,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,...,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000
mean,6.472925e+05,91694.053683,0.000110,0.234469,0.000649,0.001286,0.002121,0.014572,0.004343,0.000243,...,0.111418,0.419966,0.004401,0.175303,0.023819,0.002196,0.002916,0.027049,0.108886,0.106876
std,7.149223e+05,1088.074673,0.010509,0.423668,0.025477,0.035834,0.046004,0.119831,0.065760,0.015587,...,0.314650,0.493554,0.066192,0.380227,0.152487,0.046809,0.053922,0.162227,0.311497,0.308956
min,1.000000e+00,90001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.900000e+05,90720.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.300000e+05,91406.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.271000e+05,92688.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.000000e+07,96161.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Structure type

In [216]:
structure = temp['Structure type'].dropna()
s_structure = defaultdict(int)
for i in range(len(structure)):
    item = structure.iloc[i]
    s_structure[item] += 1
print s_structure

defaultdict(<type 'int'>, {'Tudor': 550, 'Modern': 4783, 'Loft': 330, 'Bungalow': 3778, 'Colonial': 636, 'Georgian': 37, 'Ranch': 6624, 'Pueblo style': 198, 'Contemporary': 20677, 'French': 510, 'Victorian': 82, 'Other': 51782, 'Craftsman': 2327, 'Spanish': 6663, 'Split level': 627, 'Cape cod': 2673})


In [217]:
structure_system = ['Tudor', 'Modern', 'Loft', 'Colonial', 'Georgian', 'Ranch', 'Pueblo style', 'Contemporary', 'French', 'Victorian', 'Bungalow','Craftsman', 'Spanish', 'Split level', 'Cape cod']

In [218]:
structure_df = DataFrame()
for structure_way in structure_system:
    def cleanstructure(x):
        if x == structure_way:
            return 1
        return 0
                      
    structure_new = structure.map(cleanstructure)
    structure_df['structure_' + structure_way] = structure_new

In [219]:
homes_df[structure_df.columns] = structure_df
homes_df[structure_df.columns] = homes_df[structure_df.columns].fillna(0)

# Room count

## Waiting: 异常值处理

In [220]:
room = temp['Room count'].dropna()

In [221]:
temp.loc[np.abs(temp['Room count'] - temp['Room count'] .mean()) > 5 * temp['Room count'] .std()][['Room count', 'Beds']]

,Room count,Beds
Unnamed: 0,,
23698,20.0,5
29857,19.0,5
37647,27.0,3
50993,62.0,2
61864,24.0,7
63419,20.0,3
65297,20.0,5
66505,19.0,6
78311,21.0,5


In [222]:
homes_df['room_count'] = temp['Room count']

In [223]:
to_do.append('Room count')

# Spaces

In [224]:
space = temp['Spaces'].dropna()
s_space = defaultdict(int)
for i in range(len(space)):
    items = space.iloc[i][1:-1].split(',')
    for item in items:
        s_space[item] += 1
print s_space

defaultdict(<type 'int'>, {'"Tennis Court"': 5335, '"Barbecue Area"': 23980, '"Hot Tub/Spa"': 21550, '"Fitness Center"': 6062, '"Sports Court"': 2874, '"Basketball Court"': 2567, '"Wet Bar"': 4776, '"Greenhouse"': 300, '"Pool"': 65911, '"Sauna"': 2949, '"Jetted Tub"': 5482})


In [225]:
space_system = s_space.keys()

In [226]:
space_df = DataFrame()
for space_way in space_system:
    def cleanspace(x):
        c = x[1:-1].split(',')
        for cc in c:
            if cc == space_way:
                return 1
        return 0
                      
    space_new = space.map(cleanspace)
    space_df['space_' + space_way] = space_new

In [227]:
homes_df[space_df.columns] = space_df
homes_df[space_df.columns] = homes_df[space_df.columns].fillna(0)

# School district

In [228]:
school = temp['School district'].dropna()
s_school = defaultdict(int)
for i in range(len(school)):
    item = school.iloc[i]
    s_school[item] += 1
print s_school

defaultdict(<type 'int'>, {'Brea-Olinda Unified School District': 3, 'Palm Springs USD': 2, 'El Camino': 2, 'Norwalk/LaMirada School District': 1, 'Oak Park Unified School District': 56, 'Sonoma Valley': 1, 'William S. Hart District': 1, 'Las Virgenes Schools': 1, 'Simi Valley School Dist': 1, 'El Monte': 1, 'Lompoc Unified': 1, 'Warner Unified': 3, 'Redondo Unified': 689, 'Capistrano Valley School District': 1, 'Pomona Unified Schoo': 2, 'RIO SCHOOL DISTRICT': 1, 'Tenant to Check': 1, 'Farmersville Unified School District': 1, 'OJAI UNIFIED': 3, 'Alameda': 1, 'Willits Unified School District': 1, 'Pomona': 7, 'Laguna Beach Unified': 265, 'venice': 1, 'Hermosa Beach City Elementary': 1, 'Romoland School District': 1, 'oak park unified school district': 1, 'Yuba City Unified': 1, 'Eastside School District': 1, 'Thousand Oaks': 1, 'Torrance Unified Sch': 2, 'Standard': 6, '0': 313, 'Lowell/Macy district': 1, 'Oxnard Union High School Distri': 13, 'William S. Hart Union High Sch': 5, 'BAL

In [229]:
school = temp['School district'].dropna()

In [230]:
school[:] = 1

In [231]:
homes_df['school_district'] = school

In [232]:
homes_df['school_district'] = homes_df['school_district'].fillna(0)

# Exterior material

In [233]:
exterior = temp['Exterior material'].dropna()
s_exterior = defaultdict(int)
for i in range(len(exterior)):
    items = exterior.iloc[i].split(', ')
    for item in items:
        s_exterior[item] += 1
print s_exterior

defaultdict(<type 'int'>, {'Stone': 2725, 'Metal': 226, 'Shingle': 1340, 'Wood': 14453, 'Composition': 895, 'Wood products': 823, 'Other': 28356, 'Vinyl': 438, 'Brick': 4514, 'Stucco': 49885, 'Cement concrete': 2847})


In [234]:
exterior_system = ['Stone', 'Shingle', 'Metal', 'Composition', 'Wood', 'Vinyl', 'Brick','Stucco', 'Cement concrete']

In [235]:
exterior_df = DataFrame()
for exterior_way in exterior_system:
    def cleanexterior(x):
        if x.find(exterior_way):
            return 1
        return 0
                      
    exterior_new = exterior.map(cleanexterior)
    exterior_df['exterior_' + exterior_way] = exterior_new

In [236]:
homes_df[exterior_df.columns] = exterior_df
homes_df[exterior_df.columns] = homes_df[exterior_df.columns].fillna(0)

# Lot depth

## wating: 异常值

In [237]:
lotDepth = temp['Lot depth'].dropna()
lotDepth_new = lotDepth.map(lambda x: int(x.split(' ')[0]))

In [238]:
homes_df['lotDepth'] = lotDepth_new

In [239]:
to_do.append('Lot depth')

# Lot width

## wating: 异常值

In [240]:
lotWidth = temp['Lot width'].dropna()
lotWidth_new = lotWidth.map(lambda x: int(x.split(' ')[0]))
lotWidth_new.describe()

count    77851.000000
mean        65.182914
std        108.556766
min          1.000000
25%         50.000000
50%         60.000000
75%         70.000000
max       9857.000000
Name: Lot width, dtype: float64

In [241]:
homes_df['lotWidth'] = lotWidth_new

In [242]:
to_do.append('Lot Width')

# Patio

In [243]:
patio = temp['Patio'].dropna()
s_patio = defaultdict(int)
for i in range(len(patio)):
    if patio.iloc[i].find('[') != -1:
        items = patio.iloc[i][1:-1].split(',')
        for item in items:
            s_patio[item] += 1
    else:
        items = patio.iloc[i].split(', ')
        for item in items:
            s_patio[item] += 1
print s_patio

defaultdict(<type 'int'>, {'Deck': 70, '"Porch"': 30482, 'Other': 1, 'Enclosed': 36, 'Covered': 509, '"Patio"': 67937, '"Deck"': 13502, 'Slab': 519, 'Permitted': 46, 'Balcony': 56})


In [244]:
patio_system= ['"Porch"', '"Patio"', '"Deck"']

In [245]:
patio_df = DataFrame()
for patio_way in patio_system:
    def cleanpatio(x):
        if x.find('[') != -1:
            items = x[1:-1].split(',')
            for item in items:
                if item == patio_way:
                    return 1
        return 0
                      
    patio_new = patio.map(cleanpatio)
    patio_df['patio_' + patio_way] = patio_new

In [246]:
homes_df[patio_df.columns] = patio_df
homes_df[patio_df.columns] = homes_df[patio_df.columns].fillna(0)

# water

In [247]:
water = temp['Water'].dropna()
s_water = defaultdict(int)
for i in range(len(water)):
    if water.iloc[i].find('[') != -1:
        items = water.iloc[i][1:-1].split(',')
        for item in items:
            s_water[item] += 1
    else:
        items = water.iloc[i].split(', ')
        for item in items:
            s_water[item] += 1
print s_water

defaultdict(<type 'int'>, {'Well Domestic': 1, 'District': 153, 'Irrigation Connected': 1, 'In Street': 252, 'No Water': 1, 'Other': 16, 'Public Water System': 2, 'Stock Company': 7, '"Pond"': 1653, 'Fire Hydrant': 10, 'Water Softener - Owned': 1, 'Public': 450, 'Stock/Mutual': 12, 'Meter Paid/Not In': 1, 'Paid': 25, 'City/County': 8, '"Pool"': 65381, 'Own Well': 10, 'District/Public': 718, '"Waterfront"': 1592, 'Meter on Property': 88, 'Individual Water Meter': 7, 'Irrigat Connected': 1, 'Well': 2, 'Shared Well': 5, 'Mutual Water Source': 4, 'Stream - Year Round': 1, 'Private/Mutual': 1, 'Water District': 114, 'Public District': 1, 'Private': 11, 'Hauled Water': 1, 'On Site': 3, 'Public Utility': 1225})


In [248]:
water_system = ['"Pond"', '"Pool"', '"Waterfront"']

In [249]:
water_df = DataFrame()
for water_way in water_system:
    def cleanwater(x):
        if x.find('[') != -1:
            items = x[1:-1].split(',')
            for item in items:
                if item == water_way:
                    return 1
        return 0
                      
    water_new = water.map(cleanwater)
    water_df['water_' + water_way] = water_new

In [250]:
homes_df[water_df.columns] = water_df
homes_df[water_df.columns] = homes_df[water_df.columns].fillna(0)

# HOA Fee 

In [251]:
hoa = temp['HOA Fee'].dropna()

In [252]:
re.match('\$(\d+).*','$3/mo' ).group(1)

'3'

In [253]:
def cleanHOA(x):
    matchObj = re.match('\$(\d+).*', x)
    if matchObj:
        return int(matchObj.group(1))
    elif x.isdigit():
        return int(x)
    else:
        print x
        return 0
hoa_new = hoa.map(cleanHOA)

-$150,577,609/mo


In [254]:
hoa_new[hoa_new == 0] = np.nan

In [255]:
homes_df['hoa'] = hoa_new.copy()

# View

In [256]:
view = temp['View'].dropna()
s_view = defaultdict(int)
for i in range(len(view)):
    items = view.iloc[i].split(', ')
    for item in items:
        s_view[item] += 1
print s_view

defaultdict(<type 'int'>, {'Canal': 19, 'Mountain': 22001, 'Rocks': 30, 'Hills': 20, 'City': 16897, 'Orchard/Grove': 16, 'Lake': 79, 'Green Belt': 3, 'Vineyard': 1, 'Territorial': 5530, 'Creek/Stream': 118, 'Forest': 1, 'Vincent Thomas Bridge': 28, 'Other Water': 1, 'Valley': 5, 'Other': 3, 'See Remarks': 784, 'Courtyard': 366, 'River': 9, 'Neighbor&Tree; View': 1, 'Mountain View': 2, 'Trees/Woods': 1305, 'Park': 8255, 'Trees': 2, 'City Lights': 55, 'Water': 6891, 'Panoramic': 746, 'Woods': 1, 'Forest / Woods': 2, 'Coastline': 248, 'Bluff': 35, 'Greenbelt': 5, 'Marina': 70, 'Back Bay': 38, 'Partial/Filtered': 4, 'Pier': 20, 'Desert': 49, 'Pool': 34, 'Wooded': 2, 'Harbor': 148, 'Peek View': 1, 'Ocean': 731, 'Filter': 2, 'Peek-A-Boo': 573, 'Pasture': 19, 'Landmark': 24, 'Bay': 62, 'Beach Front': 1, 'Bridge': 33, 'Catalina': 243, 'Neighborhood': 6052, 'Meadow': 22, 'Reservoir': 18, 'Other/See Comments': 1, 'Pond': 105, 'Canyon': 375, 'Setting': 7, 'White Water': 17, 'Yes': 785, 'Golf Cour

In [257]:
'''
'canal': ['Canal'],  # 21
'Lake': ['Lake'],   # 104
'River': ['River'],  # 9
'Pool':['Pool'],  # 40
'pond': ['Pond'], # 128
'Reservoir': ['Reservoir'],  # 22
'Creek/Stream': ['Creek/Stream'],# 141
'White water': ['White Water'], # 20
'''

'''
'Bay': ['Back Bay', 'Bay'],
'Ocean': ['Ocean'],
'Coastline': ['Coastline'],
'Harbor': ['Harbor'],
'island':['Catalina'],
'Marina':['Marina', 'Pier'],
'Beach Front': ['Beach Front'],
'''

'''
'Mountain': ['Mountain', 'Mountain View'],
'Rocks': ['Rocks'],
'Hills': ['Hills'],
'Valley':['Valley'],
'Canyon':['Canyon'],
'Bluff': ['Bluff'], # 悬崖
'''

'''
'Orchard': ['Orchard/Grove', 'Vineyard'],  # 18
'Park': ['Park'],
'Courtyard':['Courtyard'],
'Meadow': ['Meadow'], 
'Pasture': ['Pasture']
'Green Belt': ['Green Belt', 'Greenbelt', 'Park Or Green Belt'],
'''
view_system = {
    
    'water': ['Water', 'Canal', 'Lake', 'Creek/Stream','River', 'Pool' , 'Pond', 'Reservoir', 'White Water' ],

    'ocean': ['Back Bay', 'Bay', 'Ocean', 'Coastline', 'Harbor', 'Catalina', 'Marina', 'Pier', 'Beach Front' ],
    
    'mountain': ['Mountain', 'Mountain View','Rocks', 'Hills', 'Valley', 'Canyon',  'Bluff'],
    
    'tree': ['Trees/Woods','Trees', 'Woods','Forest / Woods','Forest','Wooded'],
    
    'green land' : ['Orchard/Grove', 'Vineyard', 'Park', 'Courtyard', 'Meadow', 'Pasture', 
                    'Green Belt', 'Greenbelt', 'Park Or Green Belt'],
    
    'City': ['City', 'City Lights'],
     
    'Territorial':['Territorial'],  # 6374
    
    'Bridge': ['Vincent Thomas Bridge', 'Bridge'],  # 29 + 35
      
    'Panoramic': ['Panoramic','Peek View'],  #854 + 1
    
    'Desert': ['Desert'],  # 54

    'Landmark': ['Landmark'], # 27
    
    'Others': ['Other','See Remarks', 'Other/See Comments']
}

In [258]:
view_df = DataFrame()
for view_way in view_system.keys():
    def cleanview(x):
        c = x.split(', ')
        for cc in c:
            if cc in view_system[view_way]:
                return 1
        return 0
                      
    view_new = view.map(cleanview)
    view_df['view_' + view_way] = view_new

In [259]:
homes_df[view_df.columns] = view_df
homes_df[view_df.columns] = homes_df[view_df.columns].fillna(0)
homes_df.describe()

,price,zipcode,"Type and Style_[""Apartment""]","Type and Style_[""Condo""]","Type and Style_[""Cooperative""]","Type and Style_[""Miscellaneous""]","Type and Style_[""Mobile / Manufactured""]","Type and Style_[""Multi Family""]","Type and Style_[""Multiple Occupancy""]","Type and Style_[""Other""]",...,view_water,view_green land,view_Landmark,view_mountain,view_tree,view_Territorial,view_ocean,view_Panoramic,view_Others,view_Desert
count,2.263280e+05,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,...,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000,226328.000000
mean,6.472925e+05,91694.053683,0.000110,0.234469,0.000649,0.001286,0.002121,0.014572,0.004343,0.000243,...,0.031936,0.038661,0.000106,0.099157,0.005801,0.024434,0.003981,0.003301,0.003482,0.000216
std,7.149223e+05,1088.074673,0.010509,0.423668,0.025477,0.035834,0.046004,0.119831,0.065760,0.015587,...,0.175830,0.192785,0.010297,0.298873,0.075945,0.154391,0.062969,0.057355,0.058903,0.014712
min,1.000000e+00,90001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.900000e+05,90720.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,5.300000e+05,91406.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.271000e+05,92688.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,8.000000e+07,96161.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Yard

In [260]:
yard = temp['Yard'].dropna()
s_yard = defaultdict(int)
for i in range(len(yard)):
    if yard.iloc[i].find('[') != -1:
        items = yard.iloc[i][1:-1].split(',')
        for item in items:
            s_yard[item] += 1
print s_yard

defaultdict(<type 'int'>, {'"Garden"': 15840, '"Fenced Yard"': 30926, '"Lawn"': 37835})


In [261]:
yard_system = s_yard.keys()

In [262]:
yard_df = DataFrame()
for yard_way in yard_system:
    def cleanyard(x):
        if x.find('[') != -1:
            items = x[1:-1].split(',')
            for item in items:
                if item == yard_way:
                    return 1
        return 0
                      
    yard_new = yard.map(cleanyard)
    yard_df['yard_' + yard_way] = yard_new

In [263]:
homes_df[yard_df.columns] = yard_df
homes_df[yard_df.columns] = homes_df[yard_df.columns].fillna(0)

# Baths

In [264]:
baths = temp.Baths.dropna()
baths_system = ['full', 'three-quarter', 'half']

In [265]:
bath_df = DataFrame()
for bath in baths_system:
    def cleanBaths(x):
        c = x.split(', ')
        for cc in c:
            cc = cc.split(' ')
            if cc[1] == bath:
                return int(cc[0])
        return 0
    baths_new = baths.map(cleanBaths)
    bath_df['bath_'+bath] = baths_new

In [266]:
homes_df[bath_df.columns] = bath_df
homes_df[bath_df.columns] = homes_df[bath_df.columns].fillna(0)

# High school

In [267]:
high = temp['High school'].dropna()

In [268]:
high[:] =1

In [269]:
homes_df['high_school'] = high

In [270]:
homes_df['high_school'].fillna(0, inplace=True)

In [271]:
'''
LaundryFeatures             0.101324
RoomType                    0.108479
StreetSuffix                0.111728
PreviousStandardStatus      0.121425
YearBuiltSource             0.121806
LotSizeSource               0.123083
ElevationUnits              0.125940
LotFeatures                 0.128335
LivingAreaUnits             0.129372
Amenities                   0.130552
WaterSource                 0.131208
HighSchoolDistrict          0.131348
StateOrProvince             0.131802
StandardStatus              0.131802
SpecialListingConditions    0.132613
CommunityFeatures           0.133250
Levels                      0.138436
Unit floor #                0.146270
CommonWalls                 0.148622
Middle school               0.151317
Materials                   0.159108
PropertyType
'''

'\nLaundryFeatures             0.101324\nRoomType                    0.108479\nStreetSuffix                0.111728\nPreviousStandardStatus      0.121425\nYearBuiltSource             0.121806\nLotSizeSource               0.123083\nElevationUnits              0.125940\nLotFeatures                 0.128335\nLivingAreaUnits             0.129372\nAmenities                   0.130552\nWaterSource                 0.131208\nHighSchoolDistrict          0.131348\nStateOrProvince             0.131802\nStandardStatus              0.131802\nSpecialListingConditions    0.132613\nCommunityFeatures           0.133250\nLevels                      0.138436\nUnit floor #                0.146270\nCommonWalls                 0.148622\nMiddle school               0.151317\nMaterials                   0.159108\nPropertyType\n'

# Materials

In [272]:
temp.Materials.value_counts()

["Double Pane/Storm Windows"]               26684
["Skylight"]                                 4602
["Double Pane/Storm Windows","Skylight"]     4034
Name: Materials, dtype: int64

In [273]:
material_system = ['"Double Pane/Storm Windows"', '"Skylight"']

In [274]:
material = temp.Materials.dropna()
material_df = DataFrame()
for material_way in material_system:
    def cleanmaterial(x):
        if x.find('[') != -1:
            items = x[1:-1].split(',')
            for item in items:
                if item == material_way:
                    return 1
        return 0
                      
    material_new = material.map(cleanmaterial)
    material_df['material_' + material_way] = material_new

In [275]:
homes_df[material_df.columns] = material_df
homes_df[material_df.columns] = homes_df[material_df.columns].fillna(0)

# Middle school

In [276]:
middle =  temp['Middle school'].dropna()
middle[:] = 1
homes_df['middle_school'] = middle

In [277]:
homes_df['middle_school'].fillna(0, inplace=True)

In [278]:
homes_df.drop_duplicates(['sold_date', 'price', 'zipcode', 'city'], inplace=True)

In [279]:
homes_df.shape


(224303, 166)

In [280]:
homes_df.to_csv('1_all_2.csv')

In [281]:
homes_df['unit_count']

Unnamed: 0
1           1.0
2           0.0
3           NaN
4           1.0
5           1.0
6         275.0
7           3.0
8           1.0
9           1.0
10          1.0
11          0.0
12          1.0
13          0.0
14          0.0
15          1.0
16        311.0
17        170.0
18          1.0
19          0.0
20          1.0
21          0.0
22          1.0
23          1.0
24         24.0
25        403.0
26          1.0
27          1.0
28          1.0
29          1.0
30          1.0
          ...  
226299      NaN
226300      1.0
226301      1.0
226302      NaN
226303      1.0
226304      1.0
226305      1.0
226306      NaN
226307      1.0
226308      NaN
226309     97.0
226310      NaN
226311      NaN
226312      1.0
226313      0.0
226314      NaN
226315      0.0
226316      1.0
226317      1.0
226318      NaN
226319      NaN
226320      1.0
226321      1.0
226322     40.0
226323      1.0
226324      NaN
226325      1.0
226326      0.0
226327      NaN
226328      1.0
Name: unit_co

# 